In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# WOE y IV
from sklearn.base import BaseEstimator, TransformerMixin

# Para KS
from scipy.stats import ks_2samp

# Configuración visual
plt.style.use('ggplot')

# Análisis Exploratorio de Datos (EDA)

En este notebook se realiza un análisis exploratorio de datos sobre el dataset de clientes de digital wallet, con el objetivo de comprender mejor los factores que afectan el Lifetime Value (LTV) de los clientes.

## Preguntas de Interés

1. **¿Cuáles son los factores más determinantes asociados a que un cliente tenga un LTV alto?**

2. **¿Cómo varía el LTV según la frecuencia de uso de la app y el nivel de ingresos?**

3. **¿Existen diferencias significativas en el LTV entre distintas ubicaciones geográficas?**

A lo largo de este notebook se responderán estas preguntas utilizando análisis estadístico, visualizaciones y técnicas vistas en clase.

In [ ]:
# Cargar datos
df = pd.read_csv('../data/digital_wallet_ltv_dataset.csv')

# Primer vistazo
print(df.shape)
print(df.info())
print(df.head())

# Chequear valores nulos y duplicados
print(df.isnull().sum())
print("Duplicados:", df.duplicated().sum())

# Tipos de datos
print(df.dtypes)

Paso 3.1: Seleccionar Solo Variables Numéricas

In [ ]:
# Seleccionar solo columnas numéricas
numeric_df = df.select_dtypes(include=[np.number])

# Matriz de correlación
corr = numeric_df.corr()

Paso 3.2: Heatmap de Correlación

### Interpretación

Observamos que las variables más correlacionadas con LTV son:

- **Total_Spent** (correlación ≈ 1): Esta variable es función directa del LTV en el dataset, por lo cual no debe ser utilizada para predicción (causa fuga de datos).
- **Avg_Transaction_Value** y **Total_Transactions** (correlación ≈ 0.66): Estas variables están fuertemente relacionadas con el LTV, pero su combinación reconstruye casi perfectamente el target.
- Otras variables como `Max_Transaction_Value`, `Min_Transaction_Value`, y las transformadas WOE de las variables categóricas tienen una correlación baja con LTV (<0.05), indicando que individualmente aportan poca señal para explicar el target.

**Conclusión:**
Las variables transaccionales (cantidad y valor de transacciones) son los factores más determinantes en el valor de vida del cliente en este dataset. Sin embargo, para un modelo predictivo honesto se deben excluir aquellas que reconstruyen el target.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlación de Variables Numéricas")
plt.show()

Paso 3.3: Gráfica de Distribución de la Variable Objetivo (LTV)

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(df['LTV'], bins=50, kde=True)
plt.title("Distribución de LTV")
plt.xlabel("LTV")
plt.show()

Paso 4.1: Crear una variable binaria para regresión logística

Vamos a transformar LTV en un objetivo binario. Por ejemplo, “1” si el LTV está arriba de la mediana y “0” si está abajo. Esto es común cuando quieres analizar churn, “buenos/malos”, etc.

In [ ]:
# Creamos una columna binaria para clasificación (LTV alto/bajo según la mediana)
df['LTV_binary'] = (df['LTV'] > df['LTV'].median()).astype(int)

Paso 4.2: Funciones para calcular WOE e IV

woe_iv() recorre cada categoría de una variable, calcula su Weight of Evidence (WoE) y suma los Information Value (IV).
	•	WoE indica si la categoría concentra más casos “buenos” (LTV bajo) o “malos” (LTV alto).
	•	IV total mide el poder predictivo de la variable:
	•	< 0.02 → irrelevante,
	•	0.02–0.1 → débil,
	•	0.1–0.3 → medio,
	•	> 0.3 → fuerte.

In [ ]:
def woe_iv(df, feature, target):
    lst = []
    categories = df[feature].unique()
    for cat in categories:
        good = ((df[feature] == cat) & (df[target] == 0)).sum()
        bad = ((df[feature] == cat) & (df[target] == 1)).sum()
        dist_good = good / (df[target] == 0).sum()
        dist_bad = bad / (df[target] == 1).sum()
        # Manejar división por cero y log(0)
        if dist_good == 0 or dist_bad == 0:
            woe = 0
        else:
            woe = np.log(dist_bad / dist_good)
        iv = (dist_bad - dist_good) * woe
        lst.append({'category': cat, 'woe': woe, 'iv': iv})
    iv_df = pd.DataFrame(lst)
    iv_total = iv_df['iv'].sum()
    return iv_df, iv_total

Paso 4.3: Calcular IV de las variables categóricas

Declara la lista categorical_vars con las variables categóricas que queremos evaluar.
	•	Recorre cada columna (for col in categorical_vars:) y llama a woe_iv(df, col, 'LTV_binary'), que devuelve:
	1.	iv_df → tabla con el WoE y el IV de cada categoría.
	2.	iv → IV total de toda la variable.
	•	Imprime el IV total de la variable y la tabla detallada (iv_df).
	•	El separador print("-"*30) solo pone una línea de guiones para que la salida sea más legible.

In [ ]:
categorical_vars = ['Location', 'Income_Level', 'App_Usage_Frequency', 'Preferred_Payment_Method']

for col in categorical_vars:
    iv_df, iv = woe_iv(df, col, 'LTV_binary')
    print(f"IV for {col}: {iv:.4f}")
    print(iv_df)
    print("-"*30)

Paso 4.4: Reemplazar variables categóricas por sus valores WOE (para modelado)

In [ ]:
# Reemplazar cada categoría por su valor WOE
for col in categorical_vars:
    iv_df, _ = woe_iv(df, col, 'LTV_binary')
    woe_map = dict(zip(iv_df['category'], iv_df['woe']))
    df[col + '_WOE'] = df[col].map(woe_map)

In [ ]:
# Variables numéricas (menos LTV y Customer_ID)
num_vars = [col for col in numeric_df.columns if col not in ['LTV', 'Customer_ID']]

# Variables WOE creadas a partir de las categóricas
woe_vars = [col + '_WOE' for col in categorical_vars]

In [ ]:
# Verifica la correlación de todas las numéricas y WOE con LTV
all_vars = num_vars + woe_vars
corrs = df[all_vars + ['LTV']].corr()['LTV'].drop('LTV')
print("Correlación de features con LTV:")
print(corrs.sort_values(ascending=False))

# Si alguna variable tiene correlación >0.98 o <-0.98, bórrala del modelado (esto indica fuga de datos)
vars_to_drop = corrs[abs(corrs) > 0.98].index.tolist()
print(f"Variables con alta correlación que serán eliminadas del modelo: {vars_to_drop}")

# Redefine X_multiple sin las variables fugadas
X_multiple = df[[col for col in (num_vars + woe_vars) if col not in vars_to_drop]]

In [ ]:
# Chequea si algún subconjunto de columnas puede predecir LTV exactamente (fuga combinada)
df['suma_transacciones'] = df['Avg_Transaction_Value'] * df['Total_Transactions']
diff = (df['suma_transacciones'] - df['LTV']).abs().sum()
print("Diferencia total entre suma_transacciones y LTV:", diff)

# También verifica el máximo de diferencia
print("Máxima diferencia fila a fila:", (df['suma_transacciones'] - df['LTV']).abs().max())

In [ ]:
# Elimina las variables que reconstruyen LTV casi perfectamente
vars_to_remove = ['Total_Transactions', 'Total_Spent','Avg_Transaction_Value']  # Total_Spent ya está fuera, agrega las otras dos

# Actualiza num_vars quitando esas columnas
num_vars = [col for col in num_vars if col not in vars_to_remove]

# Redefine X_multiple y X_logistic con las nuevas variables
X_multiple = df[num_vars + woe_vars]
X_logistic = df[num_vars + woe_vars]

Construcción de X y y, y división en train/test

### Construcción de las matrices X e y

Este bloque construye las matrices de entrada (`X`) y salida (`y`) para los tres modelos del proyecto:

- Se eliminan variables que causarían fuga de datos (`LTV`, `Total_Spent`, etc.).
- Se definen las variables numéricas útiles y las variables categóricas transformadas con WoE.
- Se crean tres conjuntos de datos:
  - `X_simple` y `y` para regresión lineal simple.
  - `X_multiple` y `y` para regresión lineal múltiple.
  - `X_logistic` y `y_logistic` para regresión logística.
- Finalmente, se realiza la división `train/test` para cada caso con un 70% de datos para entrenamiento y 30% para prueba.

In [ ]:
# Variables numéricas (menos LTV, Customer_ID, y variables que causan fuga)
vars_to_remove = ['LTV', 'Customer_ID', 'Avg_Transaction_Value', 'Total_Transactions', 'Total_Spent', 'suma_transacciones']  # suma_transacciones si la creaste

num_vars = [col for col in numeric_df.columns if col not in vars_to_remove]

# Agregamos las variables WOE
woe_vars = [col + '_WOE' for col in categorical_vars]

# X para regresión lineal simple (usando una sola variable, ejemplo Age)
X_simple = df[['Age']]
y = df['LTV']

# X para regresión múltiple (numéricas + WOE)
X_multiple = df[num_vars + woe_vars]
y = df['LTV']

# X para regresión logística (numéricas + WOE)
X_logistic = df[num_vars + woe_vars]
y_logistic = df['LTV_binary']

# División train/test
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_simple, y, test_size=0.3, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_multiple, y, test_size=0.3, random_state=42)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_logistic, y_logistic, test_size=0.3, random_state=42)

Dimensiones del Dataset

In [ ]:
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

Ejemplo de las Primeras Filas

In [ ]:
display(df.head())

Distribución de la Variable Objetivo LTV

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(df['LTV'], bins=50, kde=True)
plt.title("Distribución de LTV (Variable continua)")
plt.xlabel("LTV")
plt.show()

Distribución del Target Binario (LTV_binary)

In [ ]:
plt.figure(figsize=(4,3))
sns.countplot(x='LTV_binary', data=df)
plt.title('Distribución LTV_binario (0 = Bajo, 1 = Alto)')
plt.xlabel('LTV_binary')
plt.ylabel('Cantidad')
plt.show()

Boxplot de las variables numéricas más importantes vs LTV_binary

In [ ]:
num_plot_vars = ['Age', 'Total_Transactions', 'Avg_Transaction_Value', 'Total_Spent']
for col in num_plot_vars:
    plt.figure(figsize=(5,3))
    sns.boxplot(x='LTV_binary', y=col, data=df)
    plt.title(f'{col} vs LTV_binary')
    plt.show()

Ejemplo de DataFrame Final

In [ ]:
display(df[[*num_plot_vars, *woe_vars, 'LTV', 'LTV_binary']].head())

Eliminación de Outliers


### Eliminación de Outliers

Este bloque elimina los valores atípicos (outliers) en las variables numéricas. Para cada columna numérica (`num_vars`), se calculan la media y la desviación estándar, y se filtran los valores que estén fuera del rango de 3 desviaciones estándar (\( \mu \pm 3\sigma \)). Esto ayuda a reducir el impacto de datos extremos que podrían sesgar los modelos.

Luego, se redefine el conjunto de datos (`df_no_outliers`) y se reconstruyen las matrices de entrada (`X`) y salida (`y`) a partir de este nuevo DataFrame filtrado. Se preparan nuevamente los conjuntos para:

- `X_simple` y `y`: regresión lineal simple (solo con la variable `Age`).
- `X_multiple` y `y`: regresión lineal múltiple (variables numéricas + WoE).
- `X_logistic` y `y_logistic`: regresión logística (variables numéricas + WoE).

Finalmente, se realiza la división `train/test` para cada conjunto, usando un 70% de los datos para entrenamiento y 30% para prueba.

In [ ]:
# Opcional: Solo sobre variables numéricas, quita outliers fuera de 3 desviaciones estándar
df_no_outliers = df.copy()
for col in num_vars:
    m = df_no_outliers[col].mean()
    s = df_no_outliers[col].std()
    df_no_outliers = df_no_outliers[(df_no_outliers[col] > m - 3*s) & (df_no_outliers[col] < m + 3*s)]

print(f"Filas después de eliminar outliers: {df_no_outliers.shape[0]}")

# Redefine tus X y y a partir de df_no_outliers, no de df
X_simple = df_no_outliers[['Age']]
y = df_no_outliers['LTV']
X_multiple = df_no_outliers[num_vars + woe_vars]
X_logistic = df_no_outliers[num_vars + woe_vars]
y_logistic = df_no_outliers['LTV_binary']

# Split como antes
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_simple, y, test_size=0.3, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_multiple, y, test_size=0.3, random_state=42)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_logistic, y_logistic, test_size=0.3, random_state=42)

Regresión Lineal Simple

### Regresión Lineal Simple

En este bloque se entrena un modelo de **regresión lineal simple** utilizando únicamente la variable `Age` para predecir el LTV.

- Se entrena el modelo con `X_train_s` y `y_train_s`.
- Se generan predicciones sobre el conjunto de prueba (`X_test_s`).
- Se calculan dos métricas:
  - **RMSE**: error cuadrático medio.
  - **R² (coeficiente de determinación)**: proporción de varianza explicada por el modelo.

#### Resultados:

- **RMSE:** 429404.57
- **R²:** -0.0029

#### Interpretación:

El modelo tiene un desempeño muy pobre:
- El R² negativo indica que el modelo **predice peor que una media constante**.
- `Age` por sí sola **no explica el LTV** de los clientes.
- El gráfico muestra que los puntos están lejos de la línea de referencia (roja), lo cual confirma el bajo poder predictivo del modelo.

In [ ]:
# Modelo de regresión lineal simple
lr_simple = LinearRegression()
lr_simple.fit(X_train_s, y_train_s)
y_pred_simple = lr_simple.predict(X_test_s)

# Métricas
rmse_simple = np.sqrt(mean_squared_error(y_test_s, y_pred_simple))
r2_simple = r2_score(y_test_s, y_pred_simple)

print("Regresión Lineal Simple (usando 'Age'):")
print(f"RMSE: {rmse_simple:.2f}")
print(f"R^2: {r2_simple:.4f}")

plt.figure(figsize=(6,4))
plt.scatter(y_test_s, y_pred_simple, alpha=0.5)
plt.xlabel("LTV Real")
plt.ylabel("LTV Predicho")
plt.title("Regresión Lineal Simple - Age")
plt.plot([y_test_s.min(), y_test_s.max()], [y_test_s.min(), y_test_s.max()], 'r--')
plt.show()

Regresión Lineal Múltiple

### Regresión Lineal Múltiple

En este bloque se entrena un modelo de **regresión lineal múltiple** utilizando todas las variables numéricas (excepto las que causan fuga de datos) y las variables categóricas transformadas con WoE.

- El modelo se entrena con `X_train_m` y `y_train_m`.
- Se evalúa sobre `X_test_m` y se calculan las métricas:
  - **RMSE**: error cuadrático medio.
  - **R² (coeficiente de determinación)**: proporción de varianza explicada por el modelo.
- También se genera un gráfico de dispersión entre el LTV real y el LTV predicho.

#### Resultados:

- **RMSE:** 347832.57
- **R²:** 0.3419

#### Interpretación:

- El modelo logra explicar aproximadamente el **34% de la varianza** del LTV.
- Aunque no es excelente, este resultado es **aceptable en contextos reales** donde las variables disponibles no explican completamente el comportamiento del cliente.
- El gráfico muestra una tendencia ascendente, aunque con bastante dispersión, lo que sugiere que el modelo tiene **capacidad moderada para predecir** el LTV.

In [ ]:
# Modelo de regresión lineal múltiple
lr_multiple = LinearRegression()
lr_multiple.fit(X_train_m, y_train_m)
y_pred_multiple = lr_multiple.predict(X_test_m)

# Métricas
rmse_multiple = np.sqrt(mean_squared_error(y_test_m, y_pred_multiple))
r2_multiple = r2_score(y_test_m, y_pred_multiple)

print("Regresión Lineal Múltiple:")
print(f"RMSE: {rmse_multiple:.2f}")
print(f"R^2: {r2_multiple:.4f}")

plt.figure(figsize=(6,4))
plt.scatter(y_test_m, y_pred_multiple, alpha=0.5)
plt.xlabel("LTV Real")
plt.ylabel("LTV Predicho")
plt.title("Regresión Lineal Múltiple")
plt.plot([y_test_m.min(), y_test_m.max()], [y_test_m.min(), y_test_m.max()], 'r--')
plt.show()

Comparar mejoras en regresión múltiple

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# 1. Regresión múltiple normal (ya la tienes, solo mostramos resultados otra vez)
print("Regresión Múltiple (Normal)")
print(f"RMSE: {rmse_multiple:.2f}")
print(f"R^2: {r2_multiple:.4f}")
print("-"*30)

# 2. Regresión múltiple con escalado
scaler = StandardScaler()
X_train_m_sc = scaler.fit_transform(X_train_m)
X_test_m_sc = scaler.transform(X_test_m)

lr_scaled = LinearRegression()
lr_scaled.fit(X_train_m_sc, y_train_m)
y_pred_scaled = lr_scaled.predict(X_test_m_sc)
rmse_scaled = np.sqrt(mean_squared_error(y_test_m, y_pred_scaled))
r2_scaled = r2_score(y_test_m, y_pred_scaled)
print("Regresión Múltiple (Estandarizada)")
print(f"RMSE: {rmse_scaled:.2f}")
print(f"R^2: {r2_scaled:.4f}")
print("-"*30)

# 3. Regresión múltiple con interacciones (polinomial de grado 2)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_m_poly = poly.fit_transform(X_train_m)
X_test_m_poly = poly.transform(X_test_m)

lr_poly = LinearRegression()
lr_poly.fit(X_train_m_poly, y_train_m)
y_pred_poly = lr_poly.predict(X_test_m_poly)
rmse_poly = np.sqrt(mean_squared_error(y_test_m, y_pred_poly))
r2_poly = r2_score(y_test_m, y_pred_poly)
print("Regresión Múltiple (Interacciones)")
print(f"RMSE: {rmse_poly:.2f}")
print(f"R^2: {r2_poly:.4f}")
print("-"*30)

# 4. Regresión múltiple con logaritmo del target
# Solo si LTV no tiene valores negativos o cero
if (y_train_m > 0).all() and (y_test_m > 0).all():
    y_train_log = np.log1p(y_train_m)
    y_test_log = np.log1p(y_test_m)
    lr_log = LinearRegression()
    lr_log.fit(X_train_m, y_train_log)
    y_pred_log = lr_log.predict(X_test_m)
    rmse_log = np.sqrt(mean_squared_error(y_test_log, y_pred_log))
    r2_log = r2_score(y_test_log, y_pred_log)
    print("Regresión Múltiple (Log-LTV)")
    print(f"RMSE: {rmse_log:.2f}")
    print(f"R^2: {r2_log:.4f}")
    print("-"*30)
else:
    print("LTV tiene valores <= 0, no se puede aplicar logaritmo.")

### Comparación de Regresiones Lineales Múltiples con Distintas Transformaciones

Se evaluaron distintas variantes de regresión lineal múltiple para observar si alguna mejora el rendimiento predictivo sobre el LTV:

1. **Regresión Múltiple Normal**
   - Modelo base sin transformación adicional.
   - **RMSE:** 347,832.57
   - **R²:** 0.3419

2. **Regresión Múltiple Estandarizada**
   - Se estandarizan todas las variables con `StandardScaler`.
   - El resultado es **idéntico** al modelo normal porque no se aplicó regularización, por lo tanto **la escala no afecta el modelo**.
   - **RMSE:** 347,832.57
   - **R²:** 0.3419

3. **Regresión con Interacciones**
   - Se agregan términos de interacción entre pares de variables (productos cruzados).
   - Hubo **una leve mejora** en el rendimiento.
   - **RMSE:** 346,758.06
   - **R²:** 0.3460

4. **Regresión con Logaritmo del Target (Log-LTV)**
   - Se aplicó una transformación logarítmica a la variable objetivo `LTV` para reducir su escala y sesgo.
   - La métrica RMSE no es comparable directamente (está en escala log).
   - **RMSE:** 0.97 (log-scale)
   - **R²:** 0.3321

---

#### Conclusión:

- **Estandarizar** no tiene impacto si no se usa regularización.
- **Agregar interacciones** tiene un pequeño efecto positivo.
- **Transformar el target** con logaritmo no mejora sustancialmente el modelo.

Estas pruebas refuerzan que el dataset no tiene variables con suficiente capacidad explicativa del LTV, por lo que incluso con ingeniería de variables avanzada, las mejoras son marginales.

Regresion logistica

Escala tus variables

### Regresión Logística con Escalado

Este bloque implementa un modelo de **regresión logística** para predecir si un cliente tiene un LTV alto o bajo (`LTV_binary`), utilizando variables numéricas y WoE escaladas.

#### Pasos clave:

- Se aplica `StandardScaler` para **escalar las variables predictoras**, lo cual es recomendable para algoritmos como la regresión logística.
- Se entrena el modelo con `X_train_l_sc` y `y_train_l`, usando `LogisticRegression()` con un máximo de 2000 iteraciones para asegurar convergencia.
- Se generan predicciones de clase (`y_pred_logr`) y de probabilidad (`y_pred_logr_proba`), esta última necesaria para métricas como ROC AUC y KS.

Esta preparación es esencial para evaluar correctamente la capacidad del modelo de clasificar clientes en grupos de alto o bajo valor.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_l_sc = scaler.fit_transform(X_train_l)
X_test_l_sc = scaler.transform(X_test_l)

logr = LogisticRegression(max_iter=2000)
logr.fit(X_train_l_sc, y_train_l)
y_pred_logr = logr.predict(X_test_l_sc)
y_pred_logr_proba = logr.predict_proba(X_test_l_sc)[:, 1]

In [ ]:
# REGRESIÓN LOGÍSTICA: Predicción de LTV alto (1) vs bajo (0)
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve

# Entrenar modelo
logr = LogisticRegression(max_iter=3000)
logr.fit(X_train_l, y_train_l)
y_pred_logr = logr.predict(X_test_l)
y_pred_logr_proba = logr.predict_proba(X_test_l)[:, 1]

# Métricas
acc_logr = accuracy_score(y_test_l, y_pred_logr)
roc_auc = roc_auc_score(y_test_l, y_pred_logr_proba)

print("Regresión Logística (predice LTV alto vs bajo)")
print(f"Accuracy: {acc_logr:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# KS Statistic (Kolmogorov-Smirnov)
ks = ks_2samp(y_pred_logr_proba[y_test_l==1], y_pred_logr_proba[y_test_l==0]).statistic
print(f"KS Statistic: {ks:.4f}")

# Matriz de confusión
conf_mat = confusion_matrix(y_test_l, y_pred_logr)
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusión - Regresión Logística")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.show()

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_l, y_pred_logr_proba)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Regresión Logística')
plt.legend()
plt.show()

### Regresión Logística: Predicción de LTV Alto vs Bajo

Este bloque implementa un modelo de **regresión logística** para clasificar a los clientes según si tienen un **LTV alto (1)** o **bajo (0)**. Se evaluó su rendimiento con métricas estándar de clasificación.

#### Resultados:
- **Accuracy:** 0.7188
- **ROC AUC:** 0.8013
- **KS Statistic:** 0.5013

#### Interpretación:
- El **accuracy** de ~71.88% indica una buena capacidad de clasificación general.
- El **área bajo la curva ROC (0.80)** sugiere que el modelo discrimina bien entre clases, es decir, logra separar a los clientes de alto y bajo valor con bastante efectividad.
- El **KS (Kolmogorov-Smirnov) de 0.50** refuerza esta capacidad discriminativa, y es un valor considerado muy bueno en modelos de scoring.
- La **matriz de confusión** muestra el desempeño por clase, evidenciando que hay una cantidad equilibrada de verdaderos positivos y negativos.
- La **curva ROC** representa gráficamente la capacidad del modelo para distinguir entre clases, mostrando una mejora clara respecto a un clasificador aleatorio (línea diagonal).

**Conclusión:**
La regresión logística resulta ser el modelo más efectivo cuando el objetivo es **clasificar clientes entre alto y bajo LTV**, superando tanto al modelo lineal simple como al múltiple en métricas clave.

Bloque comparativo

In [ ]:
from scipy.stats import ks_2samp

# --- Regresión lineal simple ---
# Ya tienes: y_test_s, y_pred_simple, rmse_simple, r2_simple

# Para KS en lineales, binarizamos la predicción usando la mediana del LTV para comparar
y_pred_simple_binary = (y_pred_simple > np.median(y_test_s)).astype(int)
y_test_s_binary = (y_test_s > np.median(y_test_s)).astype(int)
ks_simple = ks_2samp(y_pred_simple_binary[y_test_s_binary == 1], y_pred_simple_binary[y_test_s_binary == 0]).statistic

# --- Regresión lineal múltiple ---
# Ya tienes: y_test_m, y_pred_multiple, rmse_multiple, r2_multiple
y_pred_multiple_binary = (y_pred_multiple > np.median(y_test_m)).astype(int)
y_test_m_binary = (y_test_m > np.median(y_test_m)).astype(int)
ks_multiple = ks_2samp(y_pred_multiple_binary[y_test_m_binary == 1], y_pred_multiple_binary[y_test_m_binary == 0]).statistic

# --- Regresión logística ---
# Ya tienes: y_test_l, y_pred_logr_proba, acc_logr, roc_auc, ks (ya calculado)
# Como RMSE de probabilidades vs clase real (no es tan interpretado, pero sirve para comparar)
rmse_logr = np.sqrt(mean_squared_error(y_test_l, y_pred_logr_proba))

# --- Tabla comparativa ---
print("="*40)
print(f"{'Modelo':<25}{'RMSE':<12}{'R^2':<12}{'KS':<12}")
print("-"*40)
print(f"{'Lineal Simple':<25}{rmse_simple:<12.2f}{r2_simple:<12.4f}{ks_simple:<12.4f}")
print(f"{'Lineal Múltiple':<25}{rmse_multiple:<12.2f}{r2_multiple:<12.4f}{ks_multiple:<12.4f}")
print(f"{'Logística':<25}{rmse_logr:<12.4f}{'N/A':<12}{ks:<12.4f}")
print("="*40)

### Comparación de Modelos Predictivos: Lineal Simple, Lineal Múltiple y Logístico

En este bloque se comparan los tres enfoques de modelado utilizados en el proyecto: **regresión lineal simple**, **regresión lineal múltiple**, y **regresión logística**. Para hacer la comparación más justa, se calculan tres métricas para cada modelo:

- **RMSE (Root Mean Squared Error):** mide el error promedio de las predicciones. En regresión logística se calcula usando la probabilidad estimada vs. clase real (no es la métrica ideal, pero es útil como referencia).
- **R² (Coeficiente de Determinación):** indica cuánta varianza del target es explicada por el modelo. No aplica para regresión logística.
- **KS (Kolmogorov-Smirnov):** mide la capacidad de un modelo para separar correctamente entre clases altas y bajas de LTV. Se binariza el target en los modelos lineales para poder compararlo.

#### Resultados

| Modelo          | RMSE        | R²       | KS     |
|-----------------|-------------|----------|--------|
| Lineal Simple   | 429404.57   | -0.0029  | 0.0000 |
| Lineal Múltiple | 347832.57   | 0.3419   | 0.5071 |
| Logística       | 0.4305      | N/A      | 0.5013 |

#### Interpretación

- **Lineal Simple:** Tiene el peor desempeño. El R² negativo y un KS de 0 indican que usar una sola variable (como la edad) no tiene poder predictivo.
- **Lineal Múltiple:** Mejora notablemente el RMSE y alcanza un R² razonable de 0.34. Además, logra discriminar bastante bien entre clientes de alto y bajo LTV (KS = 0.51).
- **Logística:** Aunque su RMSE no es comparable directamente, el **KS de 0.50** y el **ROC AUC de ~0.80** (visto anteriormente) demuestran que es el modelo más adecuado si el objetivo es **clasificación binaria** (alto vs bajo LTV).

---

**Conclusión:**
La regresión logística es el mejor modelo para **clasificación de clientes**, mientras que la regresión lineal múltiple es útil si el objetivo es **predecir un valor numérico** del LTV.

In [ ]:
import joblib

# Guarda el modelo y el scaler (si usaste StandardScaler)
joblib.dump(logr, 'modelo_logistico.pkl')
joblib.dump(scaler, 'scaler.pkl')  # si usaste uno